In [167]:
import numpy as np

In [276]:
class Conv2d(object):
    
    def __init__(self, input_channels, kernels, kernel_size, stride, padding):
        self.stride = stride
        self.padding = padding
        self.kernel_size = kernel_size
        self.w = np.random.randn(kernels, input_channels, kernel_size, kernel_size)
        self.b = np.random.randn(kernels)
        self.last_output = None
        self.last_input = None
        
    def zero_padding(self, X):
        if self.padding == "valid":
            return X
        elif self.padding == "same":
            pad = ((self.stride -1) * X.shape[0] - self.stride + self.kernel_size)/2
            pad = int(pad)
            X_padded = np.pad(X, ((0,0), (0,0), (pad, pad), (pad, pad)), mode='constant', constant_values = (0,0))
            return X_padded, pad
    
    def convolve(self, X, W, b):
        """
        Argumentos:
        X: Array numpy de entrada con dimensiones (filter_size, filter_size, n_C_prev)
        W: Array numpy con los pesos de un filtro con dimensiones (filter_size, filter_size, n_C_prev)
        b: Entero con el valor de bias de la capa actual

        Retorna:
        Z: Entero con el valor del resultado
        """

        # Multiplico elemento a elemento el valor de entrada con los pesos del filtro
        aux = X * W
        # Realizo la suma de todos los elementos
        aux = np.sum(aux)
        # Le sumo el valor del bias para obtener Z
        Z = aux + float(b)
        return Z
    
    def conv_forward(self, layer_input):
        """
        Argumentos:
        layer_input: Array numpy con los valores de entrada a la capa convolucional (batch_size, n_C_prev, n_H_prev, n_W_prev) 

        Retorna:
        Z: Array numpy con los valores de salida de la capa convolucional (batch_size, n_H, n_W, n_C)
        """
        self.last_input = layer_input

        # Obtengo las dimensiones de la entrada
        (batch_size, n_C_prev, n_H_prev, n_W_prev) = self.last_input.shape

        # Obtengo las dimensiones de los filtros
        (n_C, n_C_prev, kernel_size, kernel_size) = self.w.shape
        
        # Agrego padding con ceros al volumen de entrada
        layer_input_padded, pad = self.zero_padding(self.last_input)
        
        n_H = int((n_H_prev + 2*pad - self.kernel_size)/self.stride + 1)
        n_W = int((n_W_prev + 2*pad - self.kernel_size)/self.stride + 1)
        
        # Inicializo el volumen de salida con ceros
        self.last_output = np.zeros([batch_size, n_C, n_H, n_W])

        # Comienzo iterando sobre cada ejemplo del batch
        for i in range(batch_size):
            
            # Itero sobre el eje vertical del volumen de salida
            for h in range(n_H):
                
                # Calculo las coordenadas verticales de inicio y fin de la ventana sobre la que aplicaremos el filtro
                y_start = self.stride * h
                y_end = y_start + kernel_size
                
                # Itero sobre el eje horizontal del volumen de salida
                for w in range(n_W):
                    
                    # Calculo las coordenadas horizontales de inicio y fin de la ventana sobre la que aplicaremos el filtro
                    x_start = self.stride * w
                    x_end = x_start + kernel_size

                    # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
                    slice_from_input_padded = layer_input_padded[i, :, y_start:y_end, x_start:x_end]
        
                    # Itero sobre la cantidad de canales del volumen de salida
                    for c in range(n_C):
                        # Obtengo el valor del filtro y bias del canal correspondiente
                        kernel = self.w[c, :, :, :]
                        bias = self.b[c]

                    # Computo la operación de convolución para esta ventana
                        self.last_output[i, c, h, w] = self.convolve(slice_from_input_padded, kernel, bias)
  
        return self.last_output

    def backward(self, dl_do, X_prev):
        
        #dimensiones de la capa anterior
        (batch_size, n_H_prev, n_W_prev, n_C_prev) = X_prev.shape
        
        #dimensiones de W
        (self.kernel_size, self.kernel_size,n_C_prev, n_C) = self.w.shape
        
        #padding
        layer_input_padded, pad = self.zero_padding(X_prev)
        
        #dimensiones de dl_do
        (batch_size, n_H, n_W, n_C) = dl_do.shape
        
        #incialización con las dimensiones correctas
        dX_prev = np.zeros((batch_size, n_H_prev, n_W_prev, n_C_prev))                           
        dW = np.zeros((self.kernel_size, self.kernel_size, n_C_prev, n_C))
        db = np.zeros((1,1,1,n_C))
        
        X_prev_pad = self.zero_padding(X_prev)
        dX_prev_pad = self.zero_padding(dX_prev)
        
        for i in range(batch_size):
                
                x_prev_pad = X_prev_pad[i]
                dx_prev_pad = dX_prev_pad[i]
                
                for h in range(n_H):
                    for w in range(n_W):
                        for c in range(n_C): 
                            
                            vert_start = h
                            vert_end = vert_start + self.kernel_size
                            horiz_start = w
                            horiz_end = horiz_start + self.kernel_size
                            
                            x_slice = x_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]
                            dx_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += self.w[:,:,:,c] * dl_do[i, h, w, c]
                            dW[c,:,:,:] += x_slice * dl_do[i, h, w, c]
                            db[c,:,:,:] += dl_do[i, h, w, c]
                            
                dX_prev[i, :, :, :] = dx_prev_pad[pad:-pad, pad:-pad, :]
        
        assert(dX_prev.shape == (batch_size, n_C_prev, n_H_prev, n_W_prev))
        
        return dX_prev, dW, db           

### Prueba padding

In [277]:
batch_size = 1
input_height, input_width = (6, 6)
input_channels = 1

# Dimensiones de la convolucional
filters = 8
filter_size = 3
stride = 1
pad = "same"

test_array = np.random.randn(batch_size, input_channels, input_height, input_width)

In [278]:
conv = Conv2d(input_channels, filters, filter_size, stride, pad)

In [279]:
x_paded, pad = conv.zero_padding(test_array)

In [280]:
pad

1

In [281]:
x_paded.shape

(1, 1, 8, 8)

### Prueba Convolve

In [282]:
conv_result = conv.conv_forward(test_array)

In [283]:
print("Convolución: Result shape: {}".format(conv_result.shape))
print("Convolución: Result value: {}".format(conv_result[0,0,0,0]))

Convolución: Result shape: (1, 8, 6, 6)
Convolución: Result value: 0.8893865642213836


### Prueba Backpropagation

In [284]:
dl_do = np.random.randn(1,8,6,6)
Backprop = conv.backward(dl_do, conv_result)

ValueError: non-broadcastable output operand with shape (1,1,6,3) doesn't match the broadcast shape (1,8,6,3)